# Calculate SAVI using snappy
## Import libraries

In [1]:
import numpy
from snappy import Product, ProductData, ProductIO, ProductUtils, FlagCoding

## Read product

In [2]:
file = '/eodata/Sentinel-2/MSI/L1C/2017/09/14/S2A_MSIL1C_20170914T033531_N0205_R061_T51VUJ_20170914T033641.SAFE'

L=0.5

print("Reading...")
product = ProductIO.readProduct(file)
width = product.getSceneRasterWidth()
height = product.getSceneRasterHeight()
name = product.getName()
description = product.getDescription()
band_names = product.getBandNames()

print("Product:     %s, %s" % (name, description))
print("Raster size: %d x %d pixels" % (width, height))
print("Start time:  " + str(product.getStartTime()))
print("End time:    " + str(product.getEndTime()))
print("Bands:       %s" % (list(band_names)))


b7 = product.getBand('B4')
b10 = product.getBand('B8')

Reading...
Product:     S2A_MSIL1C_20170914T033531_N0205_R061_T51VUJ_20170914T033641, None
Raster size: 10980 x 10980 pixels
Start time:  14-SEP-2017 03:35:31.026000
End time:    14-SEP-2017 03:35:31.026000
Bands:       ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B10', 'B11', 'B12', 'view_zenith_mean', 'view_azimuth_mean', 'sun_zenith', 'sun_azimuth', 'view_zenith_B1', 'view_azimuth_B1', 'view_zenith_B2', 'view_azimuth_B2', 'view_zenith_B3', 'view_azimuth_B3', 'view_zenith_B4', 'view_azimuth_B4', 'view_zenith_B5', 'view_azimuth_B5', 'view_zenith_B6', 'view_azimuth_B6', 'view_zenith_B7', 'view_azimuth_B7', 'view_zenith_B8', 'view_azimuth_B8', 'view_zenith_B8A', 'view_azimuth_B8A', 'view_zenith_B9', 'view_azimuth_B9', 'view_zenith_B10', 'view_azimuth_B10', 'view_zenith_B11', 'view_azimuth_B11', 'view_zenith_B12', 'view_azimuth_B12']


## Calculate SAVI and write product

In [ ]:
saviProduct = Product('savi', 'savi', width, height)
saviBand = saviProduct.addBand('savi', ProductData.TYPE_FLOAT32)
saviFlagsBand = saviProduct.addBand('savi_flags', ProductData.TYPE_UINT8)
writer = ProductIO.getProductWriter('BEAM-DIMAP')

ProductUtils.copyGeoCoding(product, saviProduct)

saviFlagCoding = FlagCoding('savi_flags')
saviFlagCoding.addFlag("savi_LOW", 1, "savi below 0")
saviFlagCoding.addFlag("savi_HIGH", 2, "savi above 1")
group = saviProduct.getFlagCodingGroup()
group.add(saviFlagCoding)

saviFlagsBand.setSampleCoding(saviFlagCoding)

saviProduct.setProductWriter(writer)
saviProduct.writeHeader('snappy_savi_output.dim')

r7 = numpy.zeros(width, dtype=numpy.float32)
r10 = numpy.zeros(width, dtype=numpy.float32)

print("Writing...")

for y in range(height):
    print("processing line ", y, " of ", height)
    r7 = b7.readPixels(0, y, width, 1, r7)
    r10 = b10.readPixels(0, y, width, 1, r10)
    savi = (r10 - r7) / (r10 + r7+L)
    saviBand.writePixels(0, y, width, 1, savi)
    saviLow = savi < 0.0
    saviHigh = savi > 1.0
    saviFlags = numpy.array(saviLow + 2 * saviHigh, dtype=numpy.int32)
    saviFlagsBand.writePixels(0, y, width, 1, saviFlags)

saviProduct.closeIO()

print("Done.")

Writing...
processing line  0  of  10980
processing line  1  of  10980
processing line  2  of  10980
processing line  3  of  10980
processing line  4  of  10980
processing line  5  of  10980
processing line  6  of  10980
processing line  7  of  10980
processing line  8  of  10980
processing line  9  of  10980
processing line  10  of  10980
processing line  11  of  10980
processing line  12  of  10980
processing line  13  of  10980
processing line  14  of  10980
processing line  15  of  10980
processing line  16  of  10980
processing line  17  of  10980
processing line  18  of  10980
processing line  19  of  10980
processing line  20  of  10980
processing line  21  of  10980
processing line  22  of  10980
processing line  23  of  10980
processing line  24  of  10980
processing line  25  of  10980
processing line  26  of  10980
processing line  27  of  10980
processing line  28  of  10980
processing line  29  of  10980
processing line  30  of  10980
processing line  31  of  10980
process

processing line  320  of  10980
processing line  321  of  10980
processing line  322  of  10980
processing line  323  of  10980
processing line  324  of  10980
processing line  325  of  10980
processing line  326  of  10980
processing line  327  of  10980
processing line  328  of  10980
processing line  329  of  10980
processing line  330  of  10980
processing line  331  of  10980
processing line  332  of  10980
processing line  333  of  10980
processing line  334  of  10980
processing line  335  of  10980
processing line  336  of  10980
processing line  337  of  10980
processing line  338  of  10980
processing line  339  of  10980
processing line  340  of  10980
processing line  341  of  10980
processing line  342  of  10980
processing line  343  of  10980
processing line  344  of  10980
processing line  345  of  10980
processing line  346  of  10980
processing line  347  of  10980
processing line  348  of  10980
processing line  349  of  10980
processing line  350  of  10980
processi

processing line  583  of  10980
processing line  584  of  10980
processing line  585  of  10980
processing line  586  of  10980
processing line  587  of  10980
processing line  588  of  10980
processing line  589  of  10980
processing line  590  of  10980
processing line  591  of  10980
processing line  592  of  10980
processing line  593  of  10980
processing line  594  of  10980
processing line  595  of  10980
processing line  596  of  10980
processing line  597  of  10980
processing line  598  of  10980
processing line  599  of  10980
processing line  600  of  10980
processing line  601  of  10980
processing line  602  of  10980
processing line  603  of  10980
processing line  604  of  10980
processing line  605  of  10980
processing line  606  of  10980
processing line  607  of  10980
processing line  608  of  10980
processing line  609  of  10980
processing line  610  of  10980
processing line  611  of  10980
processing line  612  of  10980
processing line  613  of  10980
processi

processing line  851  of  10980
processing line  852  of  10980
processing line  853  of  10980
processing line  854  of  10980
processing line  855  of  10980
processing line  856  of  10980
processing line  857  of  10980
processing line  858  of  10980
processing line  859  of  10980
processing line  860  of  10980
processing line  861  of  10980
processing line  862  of  10980
processing line  863  of  10980
processing line  864  of  10980
processing line  865  of  10980
processing line  866  of  10980
processing line  867  of  10980
processing line  868  of  10980
processing line  869  of  10980
processing line  870  of  10980
processing line  871  of  10980
processing line  872  of  10980
processing line  873  of  10980
processing line  874  of  10980
processing line  875  of  10980
processing line  876  of  10980
processing line  877  of  10980
processing line  878  of  10980
processing line  879  of  10980
processing line  880  of  10980
processing line  881  of  10980
processi

processing line  1163  of  10980
processing line  1164  of  10980
processing line  1165  of  10980
processing line  1166  of  10980
processing line  1167  of  10980
processing line  1168  of  10980
processing line  1169  of  10980
processing line  1170  of  10980
processing line  1171  of  10980
processing line  1172  of  10980
processing line  1173  of  10980
processing line  1174  of  10980
processing line  1175  of  10980
processing line  1176  of  10980
processing line  1177  of  10980
processing line  1178  of  10980
processing line  1179  of  10980
processing line  1180  of  10980
processing line  1181  of  10980
processing line  1182  of  10980
processing line  1183  of  10980
processing line  1184  of  10980
processing line  1185  of  10980
processing line  1186  of  10980
processing line  1187  of  10980
processing line  1188  of  10980
processing line  1189  of  10980
processing line  1190  of  10980
processing line  1191  of  10980
processing line  1192  of  10980
processing

...

processing line  5537  of  10980
processing line  5538  of  10980
processing line  5539  of  10980
processing line  5540  of  10980
processing line  5541  of  10980
processing line  5542  of  10980
processing line  5543  of  10980
processing line  5544  of  10980
processing line  5545  of  10980
processing line  5546  of  10980
processing line  5547  of  10980
processing line  5548  of  10980
processing line  5549  of  10980
processing line  5550  of  10980
processing line  5551  of  10980
processing line  5552  of  10980
processing line  5553  of  10980
processing line  5554  of  10980
processing line  5555  of  10980
processing line  5556  of  10980
processing line  5557  of  10980
processing line  5558  of  10980
processing line  5559  of  10980
processing line  5560  of  10980
processing line  5561  of  10980
processing line  5562  of  10980
processing line  5563  of  10980
processing line  5564  of  10980
processing line  5565  of  10980
processing line  5566  of  10980
processing

processing line  5900  of  10980
processing line  5901  of  10980
processing line  5902  of  10980
processing line  5903  of  10980
processing line  5904  of  10980
processing line  5905  of  10980
processing line  5906  of  10980
processing line  5907  of  10980
processing line  5908  of  10980
processing line  5909  of  10980
processing line  5910  of  10980
processing line  5911  of  10980
processing line  5912  of  10980
processing line  5913  of  10980
processing line  5914  of  10980
processing line  5915  of  10980
processing line  5916  of  10980
processing line  5917  of  10980
processing line  5918  of  10980
processing line  5919  of  10980
processing line  5920  of  10980
processing line  5921  of  10980
processing line  5922  of  10980
processing line  5923  of  10980
processing line  5924  of  10980
processing line  5925  of  10980
processing line  5926  of  10980
processing line  5927  of  10980
processing line  5928  of  10980
processing line  5929  of  10980
processing

processing line  6266  of  10980
processing line  6267  of  10980
processing line  6268  of  10980
processing line  6269  of  10980
processing line  6270  of  10980
processing line  6271  of  10980
processing line  6272  of  10980
processing line  6273  of  10980
processing line  6274  of  10980
processing line  6275  of  10980
processing line  6276  of  10980
processing line  6277  of  10980
processing line  6278  of  10980
processing line  6279  of  10980
processing line  6280  of  10980
processing line  6281  of  10980
processing line  6282  of  10980
processing line  6283  of  10980
processing line  6284  of  10980
processing line  6285  of  10980
processing line  6286  of  10980
processing line  6287  of  10980
processing line  6288  of  10980
processing line  6289  of  10980
processing line  6290  of  10980
processing line  6291  of  10980
processing line  6292  of  10980
processing line  6293  of  10980
processing line  6294  of  10980
processing line  6295  of  10980
processing

processing line  6524  of  10980
processing line  6525  of  10980
processing line  6526  of  10980
processing line  6527  of  10980
processing line  6528  of  10980
processing line  6529  of  10980
processing line  6530  of  10980
processing line  6531  of  10980
processing line  6532  of  10980
processing line  6533  of  10980
processing line  6534  of  10980
processing line  6535  of  10980
processing line  6536  of  10980
processing line  6537  of  10980
processing line  6538  of  10980
processing line  6539  of  10980
processing line  6540  of  10980
processing line  6541  of  10980
processing line  6542  of  10980
processing line  6543  of  10980
processing line  6544  of  10980
processing line  6545  of  10980
processing line  6546  of  10980
processing line  6547  of  10980
processing line  6548  of  10980
processing line  6549  of  10980
processing line  6550  of  10980
processing line  6551  of  10980
processing line  6552  of  10980
processing line  6553  of  10980
processing

processing line  6785  of  10980
processing line  6786  of  10980
processing line  6787  of  10980
processing line  6788  of  10980
processing line  6789  of  10980
processing line  6790  of  10980
processing line  6791  of  10980
processing line  6792  of  10980
processing line  6793  of  10980
processing line  6794  of  10980
processing line  6795  of  10980
processing line  6796  of  10980
processing line  6797  of  10980
processing line  6798  of  10980
processing line  6799  of  10980
processing line  6800  of  10980
processing line  6801  of  10980
processing line  6802  of  10980
processing line  6803  of  10980
processing line  6804  of  10980
processing line  6805  of  10980
processing line  6806  of  10980
processing line  6807  of  10980
processing line  6808  of  10980
processing line  6809  of  10980
processing line  6810  of  10980
processing line  6811  of  10980
processing line  6812  of  10980
processing line  6813  of  10980
processing line  6814  of  10980
processing